In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import re
from sklearn.ensemble import RandomForestRegressor
RS=42


In [2]:
# подключаем гугл диск на котором данные
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls /content/gdrive/'My Drive'/dataset_trud/

vacs_train.csv	описание-2.xlsx


In [4]:
df_salaries500k = pd.read_csv('../content/gdrive/My Drive/dataset_trud/vacs_train.csv',sep=';')

df_salaries500k.head(20)


,index,id,name,name.lemm,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment,employment.name,schedule,schedule.name,experience,experience.name,key_skills,specializations,specializations.names,description,description.lemm,type
0,0,17686451,Менеджер по туризму,менеджер туризм,Санкт-Петербург,Санкт-Петербург,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-08-21T14:52:35+0300,40000.0,RUR,full,Полная занятость,fullDay,Полный день,between1And3,От 1 года до 3 лет,NaN,17.334 17.242 17.149 22.223 22.39 22.199,Продажи - Услуги для населения | Продажи - Пря...,Обязанности: Работа с туристами (физические и...,обязанность работа турист физический юридическ...,train
1,1,17840390,Помощник руководителя,помощник руководитель,Санкт-Петербург,unknown,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-08-21T14:28:29+0300,40000.0,RUR,full,Полная занятость,fullDay,Полный день,between1And3,От 1 года до 3 лет,NaN,4.205 4.429 6.319 6.247 2.249,Административный персонал - Персональный ассис...,Вакансия открыта в рамках отдела строительно-о...,вакансия открывать рамка отдел строительный от...,train
2,3,17665840,Аналитик,аналитик,Санкт-Петербург,unknown,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-08-19T11:57:57+0300,30000.0,RUR,full,Полная занятость,fullDay,Полный день,between1And3,От 1 года до 3 лет,NaN,1.117 1.327 1.25,"Информационные технологии, интернет, телеком -...",Обязанности: Работа в рамках отдела корпорати...,обязанность работа рамка отдел корпоративный п...,train
3,4,17914658,Авиакассир,авиакассир,Санкт-Петербург,Санкт-Петербург,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-07-29T16:17:47+0300,30000.0,RUR,full,Полная занятость,shift,Сменный график,between1And3,От 1 года до 3 лет,NaN,22.11 21.12 17.334,"Туризм, гостиницы, рестораны - Авиабилеты | Тр...",Обязанности: Бронирование и выписка авиабилет...,обязанность бронирование выписка авиабилет сис...,train
4,5,17665632,Бухгалтер,бухгалтер,Санкт-Петербург,Санкт-Петербург,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-07-18T15:39:05+0300,25000.0,RUR,full,Полная занятость,fullDay,Полный день,between3And6,От 3 до 6 лет,NaN,2.43 2.335 2.463 2.337 2.469 5.42,"Бухгалтерия, управленческий учет, финансы пред...",Обязанности: Ведение бухгалтерского и налогов...,обязанность ведение бухгалтерский налоговый уч...,train
5,6,17118641,Web-программист,Web программист,Санкт-Петербург,Санкт-Петербург,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-06-15T11:46:59+0300,45000.0,RUR,full,Полная занятость,fullDay,Полный день,between1And3,От 1 года до 3 лет,NaN,1.221 1.273 1.420 1.9 1.10,"Информационные технологии, интернет, телеком -...",Обязанности: разработка сайтов компании; инте...,обязанность разработка сайт компания интеграци...,train
6,7,16795167,Бухгалтер,бухгалтер,Санкт-Петербург,Санкт-Петербург,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-06-03T11:05:46+0300,25000.0,RUR,full,Полная занятость,fullDay,Полный день,between3And6,От 3 до 6 лет,NaN,2.43 2.335 2.463 2.337 2.469 5.42,"Бухгалтерия, управленческий учет, финансы пред...",Обязанности: Ведение бухгалтерского и налогов...,обязанность ведение бухгалтерский налоговый уч...,train
7,8,17274713,Менеджер по туризму,менеджер туризм,Санкт-Петербург,Санкт-Петербург,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-06-02T14:30:41+0300,30000.0,RUR,full,Полная занятость,fullDay,Полный день,between1And3,От 1 года до 3 лет,NaN,22.223 22.39 22.199 17.242 17.149,"Туризм, гостиницы, рестораны - Продажа туристи...",Обязанности: прием заявок от корпоративных кл...,обязанность прием заявка корпоративный клиент ...,train
8,9,17257258,Прораб отделочных работ,прораб отделочный работа,Санкт-Петербург,unknown,605490,"007, центр услуг",https://hh.ru/employer/605490,2016-06-01T11:58:44+0300,40000.0,RUR,full,Полная занятость,flexible,Гибкий график,between3And6,От 3 до 6 лет,Ответственность | Коммуникабельность,20.287 20.387,"

In [5]:
df_salaries500k.shape

(100001, 24)

In [6]:
df_salaries500k.describe()

,index,id,company.id,salary_from
count,100001.000000,1.000010e+05,1.000010e+05,100001.000000
mean,76069.849732,2.029259e+07,1.066682e+06,41048.389456
std,43601.886944,2.733736e+06,9.670595e+05,21801.077872
min,0.000000,4.140466e+06,6.000000e+00,15.000000
25%,37551.000000,1.771999e+07,8.305600e+04,28750.000000
50%,76457.000000,2.005649e+07,9.111140e+05,35000.000000
75%,114888.000000,2.369521e+07,1.880360e+06,50000.000000
max,150866.000000,2.433712e+07,3.553214e+06,400000.000000


In [7]:
df_salaries500k.columns

Index(['index', 'id', 'name', 'name.lemm', 'area.name', 'city', 'company.id',
       'company', 'company_link', 'publication_date', 'salary_from',
       'salary_currency', 'employment', 'employment.name', 'schedule',
       'schedule.name', 'experience', 'experience.name', 'key_skills',
       'specializations', 'specializations.names', 'description',
       'description.lemm', 'type'],
      dtype='object')

In [8]:
#посморим сколько у нас валют
df_salaries500k['salary_currency'].nunique()

1

хорошая новость, зарплаты у нас все в рублях. заморачиваться по переводу зарплат в других валютах нет.

In [9]:
#регулярное выражение с помощью которого будет токенизировать колонку description
TOKEN_RE = re.compile(r'[\w\d]+')  #regular expression to start with

In [10]:
def tokenize_text_simple_regex(txt, min_token_size=4):
    """ This func tokenize text with TOKEN_RE applied ealier """
    txt = txt.lower()
    all_tokens = TOKEN_RE.findall(txt)
    return [token for token in all_tokens if len(token) >= min_token_size]

In [11]:
#divide dataset to train and test

X_train, X_val, y_train, y_val=train_test_split(df_salaries500k['description.lemm'],\
                                                df_salaries500k['salary_from'], test_size=0.6,random_state=RS)

In [12]:
sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=0.95,
                                                      min_df=5)),
                             ('cls', RandomForestRegressor(n_estimators=100,max_depth=40,n_jobs=-1, \
                                                  verbose=True,random_state=RS))),
                            verbose=True)

In [13]:
sklearn_pipeline.fit(X_train,y_train);

[Pipeline] .............. (step 1 of 2) Processing vect, total=   3.9s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.7min


[Pipeline] ............... (step 2 of 2) Processing cls, total=22.1min


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 22.1min finished


In [14]:
print('Скор на трейне',sklearn_pipeline.score(X_train, y_train))


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s


Скор на трейне 0.8864883888743993


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished


In [15]:
print('Скор по валидации',sklearn_pipeline.score(X_val, y_val))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s


Скор по валидации 0.4969320918563146


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


In [16]:
from sklearn.metrics import mean_absolute_error  #for metric of our model
#let s make prediction
prediction = sklearn_pipeline.predict(X_val[-1:])
print(X_val[-1:])
print(prediction)

46017    обязанность помощь консультация покупатель тор...
Name: description.lemm, dtype: object
[25966.24406819]


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [17]:
err = mean_absolute_error(prediction, y_val.iloc[-1:])
print(err)

33.75593180752912


In [ ]:
таким образом мы предсказали зп в 25966 руб и ошиблись на 33 рубля.
Не плохо для нашала.

In [20]:
#let s see error on train and validation
print('Error on train is ',mean_absolute_error(sklearn_pipeline.predict(X_train),y_train))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s


Error on train is  5063.231661892254


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.7s finished


In [21]:
print('Error on validation is ',mean_absolute_error(sklearn_pipeline.predict(X_val),y_val))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s


Error on validation is  9309.938131664589


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.0s finished


ошибка расчитанная по всему трейну 5063 руб,
по валидации 9309 руб. Слабовато. Нужно работать!

In [22]:
#сохраним модель для испльзования на сервере
import pickle
filename = 'model_trud01.pickle'
pickle.dump(sklearn_pipeline, open(filename, 'wb'))

# This is example of code how to load saved model
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# prediction = loaded_model.predict(X_test)
# print(prediction)


In [23]:
!ls

gdrive	model_trud01.pickle  sample_data


In [ ]:
df_predictions.to_csv('sample_predictions.csv', index_label='fire_id')
#df_predictions.to_csv('data/sample_predictions.csv', index_label='fire_id')

In [ ]:
from google.colab import files
files.download('sample_predictions.csv')

In [24]:
from google.colab import files
files.download('model_trud01.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>